In [2]:
import pandas as pd

# pd.set_ max col
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("display.max_rows", 500)

# Define the path to the UMLS files
umls_dir = "../UMLS/META"

conso_file = f"{umls_dir}/MRCONSO.RRF"  # -> https://www.ncbi.nlm.nih.gov/books/NBK9685/table/ch03.T.concept_names_and_sources_file_mr/?report=objectonly
rel_file = f"{umls_dir}/MRREL.RRF"  # -> https://www.ncbi.nlm.nih.gov/books/NBK9685/table/ch03.T.related_concepts_file_mrrel_rrf/?report=objectonly
mrsty_file = f"{umls_dir}/MRSTY.RRF"

In [3]:
# Load the data files into DataFrames with specific column lengths and encodings
mrconso_columns = [
    "CUI",
    "LAT",
    "TS",
    "LUI",
    "STT",
    "SUI",
    "ISPREF",
    "AUI",
    "SAUI",
    "SCUI",
    "SDUI",
    "SAB",
    "TTY",
    "CODE",
    "STR",
    "SRL",
    "SUPPRESS",
    "CVF",
]

mrrel_columns = [
    "CUI1",
    "AUI1",
    "STYPE1",
    "REL",
    "CUI2",
    "AUI2",
    "STYPE2",
    "RELA",
    "RUI",
    "SRUI",
    "SAB",
    "SL",
    "RG",
    "DIR",
    "SUPPRESS",
    "CVF",
]

mrsty_columns = ["CUI", "TUI", "STN", "STY", "ATUI", "CVF"]


# Load MRCONSO.RRF
mrconso = pd.read_csv(
    conso_file,
    sep="|",
    header=None,
    names=mrconso_columns,
    encoding="utf-8",
    index_col=False,
)

# Load MRREL.RRF
mrrel = pd.read_csv(
    rel_file,
    sep="|",
    header=None,
    names=mrrel_columns,
    encoding="utf-8",
    index_col=False,
)

# Load MRSTY.RRF for semantic types
mrsty = pd.read_csv(
    mrsty_file,
    sep="|",
    header=None,
    names=mrsty_columns,
    encoding="utf-8",
    index_col=False,
)

/tmp/ipykernel_529797/2981246427.py:46: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  mrconso = pd.read_csv(
/tmp/ipykernel_529797/2981246427.py:56: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  mrrel = pd.read_csv(


In [4]:
# Show the first few rows of each DataFrame
print("MRCONSO.RRF")
print(mrconso.head(10))

MRCONSO.RRF
        CUI  LAT TS        LUI STT        SUI ISPREF        AUI  SAUI      SCUI     SDUI     SAB  TTY     CODE                                                STR  SRL SUPPRESS    CVF
0  C0000005  ENG  P   L0000005  PF   S0007492      Y  A26634265   NaN  M0019694  D012711     MSH  PEP  D012711                     (131)I-Macroaggregated Albumin    0        N  256.0
1  C0000005  ENG  S   L0270109  PF   S0007491      Y  A26634266   NaN  M0019694  D012711     MSH   ET  D012711                                         (131)I-MAA    0        N  256.0
2  C0000005  FRE  P   L6220710  PF   S7133957      Y  A13433185   NaN  M0019694  D012711  MSHFRE  PEP  D012711      Macroagrégats d'albumine marquée à l'iode 131    3        N    NaN
3  C0000005  FRE  S   L6215648  PF   S7133916      Y  A27488794   NaN  M0019694  D012711  MSHFRE   ET  D012711                                          MAA-I 131    3        N    NaN
4  C0000005  FRE  S   L6215656  PF   S7133956      Y  A27614225   NaN  M0

In [5]:
print("MRREL.RRF")
print(mrrel.head())

MRREL.RRF
       CUI1       AUI1 STYPE1 REL      CUI2       AUI2 STYPE2            RELA         RUI SRUI     SAB      SL  RG  DIR SUPPRESS  CVF
0  C0000005  A13433185   SCUI  RB  C0036775   A7466261   SCUI             NaN   R86000559  NaN  MSHFRE  MSHFRE NaN  NaN        N  NaN
1  C0000005  A26634265   SCUI  RB  C0036775   A0115649   SCUI             NaN   R31979041  NaN     MSH     MSH NaN  NaN        N  NaN
2  C0000039   A0016515    AUI  SY  C0000039  A11754881    AUI  translation_of  R101808683  NaN  MSHSWE  MSHSWE NaN  NaN        N  NaN
3  C0000039   A0016515    AUI  SY  C0000039  A13042554    AUI  translation_of  R193408122  NaN  MSHCZE  MSHCZE NaN  NaN        N  NaN
4  C0000039   A0016515    AUI  SY  C0000039  A13096036    AUI  translation_of   R73331672  NaN  MSHPOR  MSHPOR NaN  NaN        N  NaN


In [6]:
print("MRSTY.RRF")
print(mrsty.head())

MRSTY.RRF
        CUI   TUI           STN                                    STY        ATUI    CVF
0  C0000005  T116  A1.4.1.2.1.7        Amino Acid, Peptide, or Protein  AT17648347  256.0
1  C0000005  T121    A1.4.1.1.1                Pharmacologic Substance  AT17575038  256.0
2  C0000005  T130    A1.4.1.1.4  Indicator, Reagent, or Diagnostic Aid  AT17634323  256.0
3  C0000039  T109    A1.4.1.2.1                       Organic Chemical  AT45562015  256.0
4  C0000039  T121    A1.4.1.1.1                Pharmacologic Substance  AT17567371  256.0


In [7]:
# print all tui terms in mrsty
print(mrsty["STY"].unique())

['Amino Acid, Peptide, or Protein' 'Pharmacologic Substance'
 'Indicator, Reagent, or Diagnostic Aid' 'Organic Chemical' 'Enzyme'
 'Biologically Active Substance' 'Hazardous or Poisonous Substance'
 'Hormone' 'Nucleic Acid, Nucleoside, or Nucleotide' 'Vitamin'
 'Antibiotic' 'Immunologic Factor' 'Tissue' 'Manufactured Object'
 'Intellectual Product' 'Body Location or Region' 'Sign or Symptom'
 'Finding' 'Neoplastic Process' 'Body Part, Organ, or Organ Component'
 'Virus' 'Disease or Syndrome' 'Congenital Abnormality'
 'Functional Concept' 'Anatomical Abnormality' 'Injury or Poisoning'
 'Classification' 'Embryonic Structure' 'Pathologic Function'
 'Patient or Disabled Group' 'Health Care Related Organization'
 'Regulation or Law' 'Therapeutic or Preventive Procedure'
 'Quantitative Concept' 'Research Activity' 'Mental Process'
 'Individual Behavior' 'Fungus' 'Diagnostic Procedure'
 'Natural Phenomenon or Process' 'Inorganic Chemical'
 'Occupational Activity' 'Geographic Area' 'Medical De

In [8]:
# print unique rela
print(mrrel["RELA"].unique())

[nan 'translation_of' 'permuted_term_of' 'mapped_to' 'associated_with'
 'has_permuted_term' 'has_translation' 'has_transliterated_form'
 'measures' 'parent_of' 'form_of' 'has_component' 'transliterated_form_of'
 'inverse_isa' 'disposition_of' 'exhibited_by' 'see_from' 'see'
 'entry_combination_of' 'mapped_from' 'has_causative_agent' 'used_for'
 'use' 'isa' 'subset_includes_concept' 'has_direct_substance'
 'has_ingredient' 'has_tradename' 'mapping_qualifier_of' 'contains'
 'active_ingredient_of' 'has_active_ingredient' 'has_active_moiety'
 'has_member' 'has_basis_of_strength_substance'
 'has_precise_active_ingredient' 'tradename_of' 'may_be_prevented_by'
 'may_be_treated_by' 'has_contraindicated_drug' 'has_part'
 'physiologic_effect_of' 'mechanism_of_action_of' 'therapeutic_class_of'
 'lab_number_of' 'chemotherapy_regimen_has_component' 'has_gdc_value'
 'structural_class_of' 'has_inactive_ingredient'
 'contraindicated_class_of' 'has_free_acid_or_base_form'
 'has_contraindicated_class' '

## Side effects list

In [9]:
# Define the relevant relationships
side_effect_relationships = ["associated_with", "cause_of", "induces"]

# Filter MRREL for relevant relationships
side_effect_rels = mrrel[mrrel["RELA"].isin(side_effect_relationships)]

print("Side Effect Relationships shape:", side_effect_rels.shape)

# Display the resulting relationships
side_effect_rels_display = side_effect_rels[
    ["CUI1", "AUI1", "REL", "CUI2", "AUI2", "RELA"]
]
print(side_effect_rels_display.head())

# Additional checks and debug information
print("Unique CUIs in CUI1:", side_effect_rels["CUI1"].nunique())
print("Unique AUIs in AUI1:", side_effect_rels["AUI1"].nunique())
print("Unique RELA types:", side_effect_rels["RELA"].unique())

Side Effect Relationships shape: (166849, 16)
          CUI1      AUI1 REL      CUI2      AUI2             RELA
52    C0000039  A0049238  RO  C0523614  A0918221  associated_with
140   C0000052  A0630731  RO  C0523417  A0842968  associated_with
1219  C0000163  A0631426  RO  C0523706  A0353486  associated_with
1222  C0000163  A0631435  RO  C0373659  A0727415  associated_with
1223  C0000163  A0631435  RO  C0523732  A0979160  associated_with
Unique CUIs in CUI1: 61204
Unique AUIs in AUI1: 88493
Unique RELA types: ['associated_with' 'induces' 'cause_of']


In [10]:
# Filter MRCONSO to keep only the rows with the relevant AUIs or CUIs and English language
relevant_auis = pd.concat([side_effect_rels["AUI1"], side_effect_rels["AUI2"]]).unique()
relevant_cuis = pd.concat([side_effect_rels["CUI1"], side_effect_rels["CUI2"]]).unique()
mrconso_filtered = mrconso[
    (mrconso["AUI"].isin(relevant_auis) | mrconso["CUI"].isin(relevant_cuis))
    & (mrconso["LAT"] == "ENG")
]
print("MRCONSO filtered shape:", mrconso_filtered.shape)

MRCONSO filtered shape: (550705, 18)


In [11]:
# Create dictionaries to map AUIs and CUIs to strings
aui_to_str = dict(zip(mrconso_filtered["AUI"], mrconso_filtered["STR"]))
cui_to_str = dict(zip(mrconso_filtered["CUI"], mrconso_filtered["STR"]))

# Add the strings to the side effect relationships DataFrame using both AUIs and CUIs
side_effect_rels["AUI1_STR"] = side_effect_rels["AUI1"].map(aui_to_str)
side_effect_rels["AUI2_STR"] = side_effect_rels["AUI2"].map(aui_to_str)

# For missing AUI2_STR, try to map using CUI2
side_effect_rels["AUI2_STR"] = side_effect_rels.apply(
    lambda row: cui_to_str.get(row["CUI2"], row["AUI2_STR"]), axis=1
)
# drop the rows with missing AUI2_STR
side_effect_rels = side_effect_rels.dropna(subset=["AUI2_STR"])

print("Side Effect Relationships with strings shape:", side_effect_rels.shape)
print(side_effect_rels.head())
# # Display the resulting relationships with strings
# side_effect_rels_display = side_effect_rels[
#     ["CUI1", "AUI1", "AUI1_STR", "REL", "CUI2", "AUI2", "AUI2_STR", "RELA"]
# ]

/tmp/ipykernel_529797/3582002100.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_effect_rels["AUI1_STR"] = side_effect_rels["AUI1"].map(aui_to_str)
/tmp/ipykernel_529797/3582002100.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_effect_rels["AUI2_STR"] = side_effect_rels["AUI2"].map(aui_to_str)


Side Effect Relationships with strings shape: (166848, 18)
          CUI1      AUI1 STYPE1 REL      CUI2      AUI2 STYPE2             RELA        RUI SRUI   SAB    SL  RG  DIR SUPPRESS  CVF                           AUI1_STR                                           AUI2_STR
52    C0000039  A0049238    AUI  RO  C0523614  A0918221    AUI  associated_with  R04184949  NaN  SNMI  SNMI NaN  NaN        N  NaN     Dipalmitoylphosphatidylcholine  Dipalmitoylphosphatidylcholine measurement (pr...
140   C0000052  A0630731    AUI  RO  C0523417  A0842968    AUI  associated_with  R04221530  NaN  SNMI  SNMI NaN  NaN        N  NaN  1,4-alpha-Glucan branching enzyme  1,4-alpha-Glucan branching enzyme measurement ...
1219  C0000163  A0631426    AUI  RO  C0523706  A0353486    AUI  associated_with  R04221578  NaN  SNMI  SNMI NaN  NaN        N  NaN          17-Hydroxycorticoids, NOS   17-hydroxycorticosteroid measurement (procedure)
1222  C0000163  A0631435    AUI  RO  C0373659  A0727415    AUI  associate

/tmp/ipykernel_529797/3582002100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_effect_rels["AUI2_STR"] = side_effect_rels.apply(


In [12]:
# Define relevant T-codes for drugs and side effects
drug_stys = [
    "Pharmacologic Substance",
    "Clinical Drug",
    "Antibiotic",
    "Hormone",
    "Vitamin",
    "Immunologic Factor",
    "Enzyme",
    "Biologically Active Substance",
    "Amino Acid, Peptide, or Protein",
    "Organic Chemical",
    "Nucleic Acid, Nucleoside, or Nucleotide",
    "Hazardous or Poisonous Substance",
]
side_effect_stys = [
    "Sign or Symptom",
    "Disease or Syndrome",
    "Injury or Poisoning",
    "Pathologic Function",
    "Mental or Behavioral Dysfunction",
    "Neoplastic Process",
    "Congenital Abnormality",
    "Anatomical Abnormality",
    "Acquired Abnormality",
    "Cell or Molecular Dysfunction",
]

# Filter MRSTY for CUIs with the relevant STY for drugs and side effects
drug_cuis = mrsty[mrsty["STY"].isin(drug_stys)]["CUI"].unique()
side_effect_cuis = mrsty[mrsty["STY"].isin(side_effect_stys)]["CUI"].unique()

print("Drug CUIs shape:", drug_cuis.shape)
print("Side Effect CUIs shape:", side_effect_cuis.shape)

# Debug information
print("Sample Drug CUIs:", drug_cuis[:10])
print("Sample Side Effect CUIs:", side_effect_cuis[:10])

# Filter the side effect relationships to include only drug-to-symptom relationships
drug_side_effect_rels = side_effect_rels[
    (side_effect_rels["CUI1"].isin(drug_cuis))
    & (side_effect_rels["CUI2"].isin(side_effect_cuis))
]
print("Filtered Drug-Side Effect Relationships shape:", drug_side_effect_rels.shape)

Drug CUIs shape: (620177,)
Side Effect CUIs shape: (358502,)
Sample Drug CUIs: ['C0000005' 'C0000039' 'C0000052' 'C0000074' 'C0000084' 'C0000096'
 'C0000097' 'C0000098' 'C0000102' 'C0000103']
Sample Side Effect CUIs: ['C0000727' 'C0000729' 'C0000735' 'C0000737' 'C0000744' 'C0000754'
 'C0000762' 'C0000768' 'C0000770' 'C0000771']
Filtered Drug-Side Effect Relationships shape: (2280, 18)


In [13]:
# Display the resulting relationships with strings
drug_side_effect_rels_display = drug_side_effect_rels[
    ["CUI1", "AUI1", "AUI1_STR", "REL", "CUI2", "AUI2", "AUI2_STR", "RELA"]
]
print(
    "Drug-Side Effect Relationships with strings shape:",
    drug_side_effect_rels_display.shape,
)
print("Unique AUI1_STRs (Drugs):", drug_side_effect_rels_display["AUI1_STR"].nunique())
print(
    "Unique AUI2_STRs (Side Effects):",
    drug_side_effect_rels_display["AUI2_STR"].nunique(),
)
drug_side_effect_rels_display

Drug-Side Effect Relationships with strings shape: (2280, 8)
Unique AUI1_STRs (Drugs): 670
Unique AUI2_STRs (Side Effects): 944


,CUI1,AUI1,AUI1_STR,REL,CUI2,AUI2,AUI2_STR,RELA
4586,C0000340,A0666735,Calcidiol 1-monooxygenase,RO,C0268689,A0539920,"PSEUDOVITAMIN D-DEFICIENCY RICKETS, TYPE IA",associated_with
4587,C0000340,A0666735,Calcidiol 1-monooxygenase,RO,C0268689,A0578759,"PSEUDOVITAMIN D-DEFICIENCY RICKETS, TYPE IA",associated_with
4588,C0000340,A0666735,Calcidiol 1-monooxygenase,RO,C0268689,A0590093,"PSEUDOVITAMIN D-DEFICIENCY RICKETS, TYPE IA",associated_with
4589,C0000340,A0666735,Calcidiol 1-monooxygenase,RO,C0268689,A0590863,"PSEUDOVITAMIN D-DEFICIENCY RICKETS, TYPE IA",associated_with
5972,C0000392,A0003595,beta Alanine,RO,C0268629,A0548949,"Disorder of beta alanine, carnosine AND/OR hom...",associated_with
...,...,...,...,...,...,...,...,...
36448212,C3495435,A0718808,Hydroxymethylglutaryl-CoA lyase,RO,C0268601,A0558438,HMGCLD,associated_with
36448214,C3495435,A0718808,Hydroxymethylglutaryl-CoA lyase,RO,C1533587,A0558437,Hydroxymethylglutaric aciduria (disorder),associated_with
36649960,C3536856,A0670449,"Cephalosporin, NOS",RO,C0274511,A1050785,cephalosporin toxicity (diagnosis),associated_with
37838431,C3853624,A0022059,alpha-Amylase,RO,C0268169,A1040894,Pancreatic alpha-amylase deficiency (disorder),associated_with


In [14]:
# Create a dictionary to map each drug AUI to its list of side effects
drug_to_side_effects_aui = (
    drug_side_effect_rels_display.groupby("AUI1")
    .agg({"AUI1_STR": "first", "AUI2_STR": lambda x: list(x.dropna().unique())})
    .reset_index()
)

# Create a dictionary to map each drug CUI to its list of side effects
drug_to_side_effects_cui = (
    drug_side_effect_rels_display.groupby("CUI1")
    .agg({"AUI2_STR": lambda x: list(x.dropna().unique())})
    .reset_index()
)

# Rename columns for clarity
drug_to_side_effects_aui = drug_to_side_effects_aui.rename(
    columns={"AUI1": "Drug AUI", "AUI1_STR": "Drug Name", "AUI2_STR": "Side Effects"}
)

drug_to_side_effects_cui = drug_to_side_effects_cui.rename(
    columns={"CUI1": "Drug CUI", "AUI2_STR": "Side Effects"}
)

# Display the resulting DataFrames
print("Drug to Side Effects (AUI) DataFrame shape:", drug_to_side_effects_aui.shape)
print(drug_to_side_effects_aui.head())

print("Drug to Side Effects (CUI) DataFrame shape:", drug_to_side_effects_cui.shape)
print(drug_to_side_effects_cui.head())

Drug to Side Effects (AUI) DataFrame shape: (685, 3)
   Drug AUI         Drug Name                                       Side Effects
0  A0002809    Methyl alcohol  [Methyl alcohol causing toxic effect (disorder...
1  A0002814     Ethyl alcohol  [Acute alcoholic liver disease (disorder), alc...
2  A0003047  Methyl lomustine                     [Methyl lomustine nephropathy]
3  A0003595      beta Alanine  [Disorder of beta alanine, carnosine AND/OR ho...
4  A0003715   Sulfite oxidase  [Combined deficiency of sulfite oxidase, xanth...
Drug to Side Effects (CUI) DataFrame shape: (642, 2)
   Drug CUI                                       Side Effects
0  C0000340      [PSEUDOVITAMIN D-DEFICIENCY RICKETS, TYPE IA]
1  C0000392  [Disorder of beta alanine, carnosine AND/OR ho...
2  C0000435                               [LCATA Deficiencies]
3  C0000507  [tyrosinemia type III (diagnosis), 4-alpha-hyd...
4  C0000665  [hydroxyquinoline derivative encephalopathy (d...


In [15]:
# Check for specific drugs
specific_drugs = ["prednisolone", "ramipril", "aspirin"]

for drug in specific_drugs:
    print(f"\nSide effects for {drug}:")
    drug_aui_side_effects = drug_to_side_effects_aui[
        drug_to_side_effects_aui["Drug Name"].str.contains(drug, case=False, na=False)
    ]
    print(drug_aui_side_effects)

    drug_cui_side_effects = drug_to_side_effects_cui[
        drug_to_side_effects_cui["Drug CUI"].isin(
            drug_side_effect_rels_display[
                drug_side_effect_rels_display["AUI1_STR"].str.contains(
                    drug, case=False, na=False
                )
            ]["CUI1"].unique()
        )
    ]
    print(drug_cui_side_effects)


Side effects for prednisolone:
Empty DataFrame
Columns: [Drug AUI, Drug Name, Side Effects]
Index: []
Empty DataFrame
Columns: [Drug CUI, Side Effects]
Index: []

Side effects for ramipril:
Empty DataFrame
Columns: [Drug AUI, Drug Name, Side Effects]
Index: []
Empty DataFrame
Columns: [Drug CUI, Side Effects]
Index: []

Side effects for aspirin:
Empty DataFrame
Columns: [Drug AUI, Drug Name, Side Effects]
Index: []
Empty DataFrame
Columns: [Drug CUI, Side Effects]
Index: []


In [16]:
# Filter MRCONSO to keep only the English language entries
mrconso_filtered = mrconso[mrconso["LAT"] == "ENG"]

# Find AUIs and CUIs associated with "prednisolone"
prednisolone_auis = mrconso_filtered[
    mrconso_filtered["STR"].str.contains("prednisolone", case=False, na=False)
]["AUI"].unique()
prednisolone_cuis = mrconso_filtered[
    mrconso_filtered["STR"].str.contains("prednisolone", case=False, na=False)
]["CUI"].unique()

print(f"Prednisolone AUIs: {prednisolone_auis}")
print(f"Prednisolone CUIs: {prednisolone_cuis}")

Prednisolone AUIs: ['A27059094' 'A27060898' 'A30134867' ... 'A36028236' 'A36022870'
 'A36186578']
Prednisolone CUIs: ['C0005308' 'C0011705' 'C0011777' 'C0016298' 'C0016369' 'C0025815'
 'C0025819' 'C0032949' 'C0032950' 'C0040864' 'C0044955' 'C0049623'
 'C0050236' 'C0050414' 'C0051113' 'C0057181' 'C0057598' 'C0058004'
 'C0065847' 'C0066174' 'C0066428' 'C0066429' 'C0066430' 'C0068500'
 'C0071836' 'C0071838' 'C0071839' 'C0071840' 'C0071842' 'C0071843'
 'C0071844' 'C0071845' 'C0071846' 'C0078042' 'C0078143' 'C0078165'
 'C0078413' 'C0100397' 'C0119180' 'C0138273' 'C0138274' 'C0138275'
 'C0174938' 'C0214080' 'C0251599' 'C0257583' 'C0278794' 'C0279051'
 'C0279109' 'C0279263' 'C0279265' 'C0279341' 'C0279343' 'C0279351'
 'C0279386' 'C0279441' 'C0279475' 'C0279487' 'C0280000' 'C0280024'
 'C0280075' 'C0280095' 'C0280463' 'C0280640' 'C0280643' 'C0280715'
 'C0280807' 'C0280824' 'C0280859' 'C0280861' 'C0280909' 'C0281313'
 'C0281314' 'C0281366' 'C0281400' 'C0281452' 'C0281465' 'C0281482'
 'C0281483' 

In [17]:
# Filter MRSTY for the CUIs associated with "prednisolone"
prednisolone_tuis = mrsty[mrsty["CUI"].isin(prednisolone_cuis)]

print(f"Prednisolone TUIs shape: {prednisolone_tuis.shape}")
print(prednisolone_tuis.head())

# Display the unique TUIs and their associated semantic types
unique_tuis = prednisolone_tuis["TUI"].unique()
unique_stys = prednisolone_tuis["STY"].unique()
print(f"Unique TUIs: {unique_tuis}")
print(f"Unique Semantic Types (STYs): {unique_stys}")

Prednisolone TUIs shape: (982, 6)
           CUI   TUI           STN                      STY         ATUI     CVF
3440  C0005308  T109    A1.4.1.2.1         Organic Chemical   AT58733215  4352.0
3441  C0005308  T121    A1.4.1.1.1  Pharmacologic Substance   AT17565972  4352.0
3442  C0005308  T125  A1.4.1.1.3.2                  Hormone   AT19259756  4352.0
7859  C0011705  T109    A1.4.1.2.1         Organic Chemical  AT102421589  4352.0
7860  C0011705  T121    A1.4.1.1.1  Pharmacologic Substance   AT17566216  4352.0
Unique TUIs: ['T109' 'T121' 'T125' 'T061' 'T116' 'T192' 'T200' 'T074' 'T046' 'T201'
 'T047' 'T114' 'T130' 'T203' 'T059' 'T195' 'T129' 'T073']
Unique Semantic Types (STYs): ['Organic Chemical' 'Pharmacologic Substance' 'Hormone'
 'Therapeutic or Preventive Procedure' 'Amino Acid, Peptide, or Protein'
 'Receptor' 'Clinical Drug' 'Medical Device' 'Pathologic Function'
 'Clinical Attribute' 'Disease or Syndrome'
 'Nucleic Acid, Nucleoside, or Nucleotide'
 'Indicator, Reagent, or 